<a href="https://colab.research.google.com/github/SaiNitishV/Algorithms-Project-TeamC/blob/main/Used_Cars_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# 1. Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
import tensorflow as tf


# 2. Load the training and testing datasets
train_df = pd.read_csv('/content/drive/MyDrive/train.csv', encoding='latin1')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv', encoding='latin1')

**EDA**

In [21]:
# Shape of the dataset
print("Shape of the dataset:", train_df.shape)

# Data types of each column
print("Data types:\n", train_df.dtypes)

# Basic statistics (numerical columns)
print("Basic statistics:\n", train_df.describe())

# Basic statistics (categorical columns)
print("Categorical column stats:\n", train_df.describe(include=['object']))

# Check for missing values
print("Missing values per column:\n", train_df.isnull().sum())

Shape of the dataset: (188498, 15)
Data types:
 id                float64
brand              object
model              object
model_year        float64
Age_of_vehicle      int64
milage            float64
fuel_type          object
Horse power        object
Engine Type        object
transmission       object
ext_col            object
int_col            object
accident          float64
clean_title       float64
price             float64
dtype: object
Basic statistics:
              id  model_year  Age_of_vehicle    milage  accident  clean_title  \
count 187579.00   187579.00       188498.00 187579.00 185150.00    187579.00   
mean   94259.66     2015.87           17.96  65644.71      0.22         0.89   
std    54429.33        5.59          140.52  49743.74      0.42         0.32   
min        0.00     1974.00            0.00    100.00      0.00         0.00   
25%    47118.50     2013.00            4.00  24115.00      0.00         1.00   
50%    94261.00     2017.00            7.00  5761

In [22]:
# Identify rows that contain 'â“' or similar patterns
problematic_rows_mask = train_df.apply(lambda x: x.str.contains('â', na=False).any(), axis=1)

In [23]:
# Identify rows that contain 'â“' or similar patterns
problematic_rows = train_df[train_df.apply(lambda x: x.str.contains('â', na=False).any(), axis=1)]

In [24]:
problematic_rows

,id,brand,model,model_year,Age_of_vehicle,milage,fuel_type,Horse power,Engine Type,transmission,ext_col,int_col,accident,clean_title,price
5,5.00,Audi,A6 2.0T Sport,2018.00,6,40950.00,Gasoline,252.0HP,2.0L 4 Cylinder Engine,A/T,White,â,0.00,1.00,29950.00
28,28.00,Toyota,Tacoma PreRunner,2008.00,16,244200.00,Gasoline,236.0HP,4.0L V6 Cylinder Engine,A/T,Silver,â,1.00,1.00,3499.00
127,128.00,Volvo,S80 3.2,2001.00,23,90050.00,Gasoline,235.0HP,3.2L Straight 6 Cylinder Engine,A/T,White,â,0.00,1.00,15900.00
157,158.00,Audi,Q7 3.0T Premium Plus,2017.00,7,70582.00,Gasoline,333.0HP,3.0L V6 Cylinder Engine,8-Speed A/T,White,â,0.00,1.00,24700.00
213,214.00,Jaguar,XJ6 Vanden Plas,2004.00,20,170000.00,Gasoline,300.0HP,4.2L 8 Cylinder Engine,6-Speed A/T,Gold,â,1.00,1.00,4900.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188424,188459.00,Mazda,MX-5 Miata Shinsen,2000.00,24,165000.00,Gasoline,158.0HP,2.0L 4 Cylinder Engine,A/T,Black,â,0.00,1.00,4000.00
188440,188475.00,Toyota,Matrix XR,2000.00,24,185000.00,Gasoline,130.0HP,1.8L 4 Cylinder Engine,4-Speed A/T,Gray,â,1.00,1.00,2500.00
188441,188476.00,Mercedes-Benz,S-Class S 580 4MATIC,2020.00,4,88949.00,Gasoline,4.0L,0L V8 32V GDI DOHC,8-Speed Automatic,â,Black,0.00,1.00,267950.00
188452,188487.00,Mercedes-Benz,SL-Class SL500 Roadster,2003.00,21,140000.00,Gasoline,302.0HP,5.0L 8 Cylinder Engine,A/T,Silver,â,0.00,1.00,10000.00


In [7]:
problematic_rows.to_csv("Noise_Data.csv")

In [25]:
print(problematic_rows_mask.sum())
print(train_df.shape)
print(train_df.isnull().sum())

4863
(188498, 15)
id                 919
brand              919
model              919
model_year         919
Age_of_vehicle       0
milage             919
fuel_type          919
Horse power        919
Engine Type       1839
transmission       919
ext_col            919
int_col            919
accident          3348
clean_title        919
price              919
dtype: int64


In [26]:
train_df = train_df[~problematic_rows_mask].copy()
print(train_df.shape)
train_df.reset_index(drop=True, inplace=True)
train_df.ffill(inplace=True)
print(train_df.isnull().sum())

(183635, 15)
id                0
brand             0
model             0
model_year        0
Age_of_vehicle    0
milage            0
fuel_type         0
Horse power       0
Engine Type       0
transmission      0
ext_col           0
int_col           0
accident          0
clean_title       0
price             0
dtype: int64


In [10]:
train_df.to_csv("df_train.csv")

In [27]:
# 4. Data Preprocessing
# Separate features (X) and target variable (y) for training
# print(train_df.shape)
# train_df = train_df.remove(problematic_rows)
X_train = train_df.drop(['price', 'id', 'Horse power'], axis=1)
y_train = train_df['price']
print(train_df.columns)
# Select categorical and numerical columns
categorical_cols = ['brand', 'model', 'fuel_type', 'Engine Type', 'transmission', 'ext_col', 'int_col']
numerical_cols = ['Age_of_vehicle', 'milage', 'accident', 'clean_title']

# Define preprocessing for numerical data (scaling) and categorical data (one-hot encoding)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

Index(['id', 'brand', 'model', 'model_year', 'Age_of_vehicle', 'milage',
       'fuel_type', 'Horse power', 'Engine Type', 'transmission', 'ext_col',
       'int_col', 'accident', 'clean_title', 'price'],
      dtype='object')


In [28]:
# 5. Model Training with a Pipeline
# Create a pipeline with preprocessing and model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators=100, random_state=42,
                            tree_method='hist', device='cuda'))
])

# Fit the model on training data
model_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:46:54] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [12:46:54] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age_of_vehicle', 'milage',
                                                   'accident', 'clean_title']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['brand', 'model',
                                                   'fuel_type', 'Engine Type',
                                                   'transmission', 'ext_col',
                                                   'int_col'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_byle...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [13]:
# Identify rows that contain 'â“' or similar patterns
problematic_rows_mask = test_df.apply(lambda x: x.str.contains('â', na=False).any(), axis=1)
# print(problematic_rows_mask.head(10))
# print(problematic_rows.size)
print(test_df.shape)
test_df = test_df[~problematic_rows_mask].copy()
print(test_df.shape)
test_df.reset_index(drop=True, inplace=True)
test_df.ffill(inplace=True)

# 6. Prepare the testing dataset
# Data Cleaning for testing dataset
test_df.fillna(method='ffill', inplace=True)  # Fill any remaining NaNs

# Check for any remaining missing values in the testing set
# print("Testing data missing values:\n", test_df.isnull().sum())

# Separate features for testing
X_test = test_df.drop(['id', 'Horse power'], axis=1)  # Drop 'id' since it's irrelevant

(125690, 14)
(122000, 14)


<ipython-input-13-b52b6de4eb3b>:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_df.fillna(method='ffill', inplace=True)  # Fill any remaining NaNs


In [31]:
problematic_rows.columns

Index(['id', 'brand', 'model', 'model_year', 'Age_of_vehicle', 'milage',
       'fuel_type', 'Horse power', 'Engine Type', 'transmission', 'ext_col',
       'int_col', 'accident', 'clean_title', 'price'],
      dtype='object')

In [32]:
# Identify rows that contain 'â“' or similar patterns
problematic_rows_mask = test_df.apply(lambda x: x.str.contains('â', na=False).any(), axis=1)

# Print shape before replacing values
print("Shape before replacing problematic values:", test_df.shape)

# Loop through each column in the test_df
for col in test_df.columns:
    if test_df[col].dtype != 'object':  # Ensure it's a numerical column
        # Find the median of the column
        median_value = test_df[col].median()

        # Replace problematic rows in that column with the median value
        test_df.loc[problematic_rows_mask, col] = test_df.loc[problematic_rows_mask, col].replace(to_replace='â', value=median_value)

    elif test_df[col].dtype == 'object':  # Check if the column is categorical (object type)
      # Replace problematic values (e.g., 'â“') with a placeholder or mode
      test_df[col] = test_df[col].replace({'â“': 'Unknown', 'â€“': '-', 'â€™': "'", 'â€œ': '"', 'â€': '"'}, regex=True)

      # Alternatively, fill NaN values in categorical columns
      most_frequent_value = test_df[col].mode()[0]  # Get the most frequent value (mode)
      test_df[col].fillna(most_frequent_value, inplace=True)

# Reset index and fill remaining missing values
test_df.reset_index(drop=True, inplace=True)
test_df.ffill(inplace=True)

# Print shape after replacing values
print("Shape after replacing problematic values:", test_df.shape)

# Check for any remaining missing values in the testing set
print("Testing data missing values:\n", test_df.isnull().sum())

# Separate features for testing
X_test = test_df.drop(['id', 'Horse power'], axis=1)  # Drop 'id' since it's irrelevant

Shape before replacing problematic values: (125690, 14)
Shape after replacing problematic values: (125690, 14)
Testing data missing values:
 id                0
brand             0
model             0
model_year        0
Age_of_vehicle    0
milage            0
fuel_type         0
Horse power       0
Engine Type       0
transmission      0
ext_col           0
int_col           0
accident          0
clean_title       0
dtype: int64


In [33]:
# 7. Predicting prices for the testing dataset
y_pred = model_pipeline.predict(X_test)

# Output the predictions
predictions_df = pd.DataFrame({'Predicted Price': y_pred})

predictions_df.to_csv('/content/drive/MyDrive/predicted_prices.csv', index=False)

# Print predictions
print(predictions_df.head())

predictions_df.to_csv("predictions_df.csv")

   Predicted Price
0         17319.98
1         72911.44
2         53541.71
3         28358.55
4         29201.29


In [34]:
# Set pandas to display floats with 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

ids = test_df['id']

# Ensure y_pred is rounded correctly
y_pred_rounded = np.round(y_pred, 2)

# Output the predictions along with the 'id' column
predictions_df = pd.DataFrame({
    'id': ids,
    'Predicted Price': y_pred_rounded
})

# Display the first few rows of the predictions DataFrame
print(predictions_df.head())

# Optionally, save the predictions
predictions_df.to_csv("predicted_prices.csv", index=False)

       id  Predicted Price
0  188533         17319.98
1  188534         72911.44
2  188535         53541.70
3  188536         28358.56
4  188537         29201.29
